In [ ]:
import torch
from model.UNet3D import UNet3D
from utils import load_config, plot_overlay, plot_slices, plot_test
from dataloader import Dataset
from torch.utils.data import DataLoader, random_split
from train import Trainer, Tester
from logger import Logger
from postprocessing import calc_volumes

print(torch.__version__) 
print(torch.cuda.is_available())

if __name__ == "__main__":  # must be enabled for num_workers > 0
    config = load_config("config.json")
    dataset = Dataset(config)

    train_set, val_set, test_set = random_split(dataset=dataset,
                                                lengths=[0.7,0.2,0.1], 
                                                generator=torch.Generator().manual_seed(42))

    train_loader = DataLoader(dataset=train_set, 
                            batch_size=config["trainer"]["batch_size"],
                            shuffle=True
                            )
    
    val_loader = DataLoader(dataset=val_set, 
                            batch_size=1
                            )
    
    test_loader = DataLoader(dataset=test_set,
                            batch_size=1
                            )
    
    print(f"Train: {len(train_loader)}\nVal: {len(val_loader)}\nTest: {len(test_loader)}")

    model = UNet3D(in_channels=1, num_classes=2)
    myLogger = Logger(config=config, save=True)
    model.cuda()
    model.load_state_dict(torch.load(R"saved\20240227_172605 320x320x16 150e 0.0005 aug\weights.pth"))

In [ ]:
pd_data = calc_volumes(test_set, model)

In [ ]:
import pandas as pd
df = pd.DataFrame(pd_data, columns=['Filename', 'Volume [mm^3]', 'Volume [L]', 'DSC (pre-resample)', 'DSC'])
df

In [ ]:
from postprocessing import Volume
pred = model(test_set[0][0].unsqueeze(0).unsqueeze(0).cuda())
vobj = Volume(image=test_set[0][0], 
              mask=test_set[0][1],
              prediction=pred,
              header=test_set[0][2])
vobj.get_volume()

In [ ]:
test_set[0][2]

In [ ]:
640*640*17*(0.625*0.625*24)

In [ ]:
160*160*16*(2.5*2.5*25.5)

In [ ]:
import numpy as np
voxel_volume = vobj.get_new_spacings(test_set[0][2], test_set[0][1].shape)
print(voxel_volume)
print()
np.count_nonzero(test_set[0][1]==1)*6*(2.5*2.5*4)